# Projeto Final: Sistemas de Recomendação
<hr>

#### Implementação de funções usadas em sistemas de recomendação <br>

In [1]:
# importando bibliotecas necessarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, hamming, cosine

In [2]:
# gerando a matriz de avaliações conforme instruções do projeto
ratings = {'I1': [4.0, 5.0, 0.0, 4.1, 1.0],
           'I2': [0.0, 4.5, 0.0, 3.0, 4.0],
           'I3': [0.0, 4.0, 0.0, 0.0, 0.0],
           'I4': [4.7, 0.0, 1.5, 4.9, 2.5],
           'I5': [1.0, 0.0, 5.0, 0.0, 3.8],
           'I6': [0.0, 0.0, 4.0, 0.0, 1.0],
           'I7': [0.0, 0.0, 0.0, 3.0, 5.0]}
ratings_df = pd.DataFrame(ratings, index=['U1', 'U2', 'U3', 'U4', 'U5'], dtype=float)
ratings_df

,I1,I2,I3,I4,I5,I6,I7
U1,4.0,0.0,0.0,4.7,1.0,0.0,0.0
U2,5.0,4.5,4.0,0.0,0.0,0.0,0.0
U3,0.0,0.0,0.0,1.5,5.0,4.0,0.0
U4,4.1,3.0,0.0,4.9,0.0,0.0,3.0
U5,1.0,4.0,0.0,2.5,3.8,1.0,5.0


### Pergunta 1:
Qual o valor da similaridade de cossenos entre os usuários U1 e U2?

In [3]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U2'].values

def cosine_similarity(x,y):
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

print(round(cosine_similarity(x,y),2))

0.41


### Pergunta 2:
Qual o valor da similaridade de cossenos entre os usuários U1 e U3?

In [4]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U3'].values

print(round(cosine_similarity(x,y),2))

0.29


### Pergunta 3:
Qual o valor da similaridade de cossenos entre os usuários U1 e U4?

In [5]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U4'].values

print(round(cosine_similarity(x,y),2))

0.82


### Pergunta 4:
Qual o valor da similaridade de cossenos entre os usuários U1 e U5?

In [6]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U5'].values

print(round(cosine_similarity(x,y),2))

0.39


### Pergunta 5:
Qual o valor da similaridade de cossenos centralizada entre os usuários U1 e U2?

In [7]:
def array_centering(v: np.ndarray):
  v = v.copy()
  non_zeros = v > 0
  v[non_zeros] = v[non_zeros] - np.mean(v[non_zeros]) + 1e-6
  return v

def centered_cosine_similarity(x,y):
  x = array_centering(x)
  y = array_centering(y)

  centered_cosine_sim = cosine_similarity(x,y)
  return centered_cosine_sim

x = ratings_df.loc['U1'].values
y = ratings_df.loc['U2'].values

print(round(centered_cosine_similarity(x,y),2))

0.2


### Pergunta 6:
Qual o valor da similaridade de cossenos centralizada entre os usuários U1 e U3?

In [8]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U3'].values

print(round(centered_cosine_similarity(x,y),2))

-0.89


### Pergunta 7:
Qual o valor da similaridade de cossenos centralizada entre os usuários U1 e U4?

In [9]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U4'].values

print(round(centered_cosine_similarity(x,y),2))

0.44


### Pergunta 8:
Qual o valor da similaridade de cossenos centralizada entre os usuários U1 e U5?

In [10]:
x = ratings_df.loc['U1'].values
y = ratings_df.loc['U5'].values

print(round(centered_cosine_similarity(x,y),2))

-0.39


### Pergunta 9:
Considerando a similaridade de cossenos centralizada, qual o usuário é mais similar ao usuário U1?

In [11]:
x = ratings_df.loc['U1'].values
a = ratings_df.loc['U2'].values
b = ratings_df.loc['U3'].values
c = ratings_df.loc['U4'].values
d = ratings_df.loc['U5'].values

print('User 2: ', round(centered_cosine_similarity(x,a),2))
print('User 3: ',round(centered_cosine_similarity(x,b),2))
print('User 4: ',round(centered_cosine_similarity(x,c),2))
print('User 5: ',round(centered_cosine_similarity(x,d),2))

User 2:  0.2
User 3:  -0.89
User 4:  0.44
User 5:  -0.39


### Pergunta 10:
Considerando a similaridade de cossenos centralizada, qual o segundo usuário mais similar ao usuário U1?

In [12]:
print('User 2: ', round(centered_cosine_similarity(x,a),2))

User 2:  0.2


### Pergunta 11:
Considerando uma filtragem colaborativa User-User, com agregação pela média simples e número de vizinhos igual a 2, qual a predição para a avaliação do usuário U1 ao item I2?

In [13]:
def estimate_rating(ratings_df,
                    user_index=0,
                    item_index=0,
                    k=2,
                    similarity=centered_cosine_similarity,
                    aggregation='mean'):
  item_rating = ratings_df.iloc[item_index].values

  items_indexes = np.array([i for i in range(ratings_df.shape[0]) if i != item_index])
  similarities = np.array([similarity(x=item_rating, y=ratings_df.iloc[i].values) for i in items_indexes])

  items_rated = np.where(ratings_df.iloc[items_indexes, user_index]> 0)[0]
  sorted_similarities = np.argsort(-1*similarities[items_rated])
  k_closest = items_indexes[items_rated][sorted_similarities[:k]]

  ratting_k_closest = ratings_df.iloc[k_closest, user_index]
  if aggregation == 'mean':
    prediction = np.mean(ratting_k_closest)
  elif aggregation == 'wmean':
    similarities_k_closest = similarities[items_rated][sorted_similarities[:k]]
    prediction = np.dot(np.abs(similarities_k_closest),
                        ratting_k_closest)/np.sum(np.abs(similarities_k_closest))
  else:
    raise ValueError('Invalid aggregation function')

  prediction = prediction if not np.isnan(prediction) else 3

  print(f"Predicao do user {user_index}, item{item_index},"\
        f"k={k}, aggregation {aggregation} é: {prediction:.2f}")
  return prediction

prediction = estimate_rating(ratings_df, user_index=1, item_index=2, k=2, aggregation='mean')

Predicao do user 1, item2,k=2, aggregation mean é: 4.25


### Pergunta 12:
Considerando uma filtragem colaborativa User-User, com agregação pela média simples e número de vizinhos igual a 2, qual a predição para a avaliação do usuário U1 ao item I1? Suponha que a avaliação (U1, I1) seja desconhecida.

In [14]:
prediction = estimate_rating(ratings_df, user_index=1, item_index=1, k=2, aggregation='mean')

Predicao do user 1, item1,k=2, aggregation mean é: 3.50
